# Flask: WebApp using sqlite3

Prerequistes:
*    pip install flask
*    pip install flask_sqlalchemy
*    pip install selenium
*    pip install webdriver_manager

Create a workspace for our project

In [ ]:
stty -echo
mkdir workspace3
cd workspace3

Create a basic webapp with a database

In [ ]:
# create new app
cat << EOF > database_app.py
import os
from flask import Flask, render_template, request, url_for, redirect
from flask_sqlalchemy import SQLAlchemy

from sqlalchemy.sql import func


basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] =\
        'sqlite:///' + os.path.join(basedir, 'database.db')

db = SQLAlchemy(app)

class Match(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    homeTeam = db.Column(db.String(100), nullable=False)
    homeScore = db.Column(db.Integer)
    awayTeam = db.Column(db.String(100), nullable=False)
    awayScore = db.Column(db.Integer)

    def __init__(self, home, score1, away, score2):
        self.homeTeam = home
        self.homeScore = score1
        self.awayTeam = away
        self.awayScore = score2

    def play(self, home, score1, away, score2):
        self.homeTeam = home
        self.homeScore = score1
        self.awayTeam = away
        self.awayScore = score2
        
    def __str__(self):
        return f"{self.homeTeam} {self.homeScore}-{self.awayScore} {self.awayTeam}"

@app.route('/')
def index():
    matches = Match.query.all()
    return render_template('index.html', matches=matches)

EOF

Run the Flask shell program to create the in-memory database

In [ ]:
export FLASK_APP=database_app
flask shell << EOF
from database_app import db, Match
db.drop_all()
db.create_all()
EOF

Now run the Flask shell program to populate the database with some example entries

In [ ]:
export FLASK_APP=database_app
flask shell << EOF

def playMatch(home, score1, away, score2):
    m = Match()
    m.play(home, score1, away, score2)
    return m

m1 = Match("Red", 2, "Blue", 1)
m2 = Match("Green", 5, "White", 0)
m3 = Match("Red", 3, "Green", 1)

db.session.add(m1)
db.session.add(m2)
db.session.add(m3)
db.session.commit()
Match.query.all()
EOF

Create a `base` template using the `jinja2` template engine.   Our actual templates will inherit from this template.

In [ ]:
mkdir -p templates
cat << EOF > templates/base.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <link rel="stylesheet" type= "text/css" href= "{{ url_for('static',filename='styles/mystyles.css') }}"    
</head>
<body>
    <h1>{% block title %} {% endblock %}</h1>
    <div>
        {% block contents %} {% endblock %}
    </div>
</body>
</html>
EOF

Now create an real template.  
The base template defines the basic structure, but we need to define the `block title` and `block content` sections here.

In [ ]:
mkdir -p templates
cat << EOF > templates/index.html
{% extends 'base.html' %}
{% block title %}
RESULTS
{% endblock %}
{% block contents %}
    <table>
    {% for match in matches %}
        <tr><td>{{ match }}</td></tr>
    {% endfor %}
    </table>
    <!-- this link will be used later -->
    <a href="http://localhost:8000/create">create</a>
{% endblock %}
EOF

Create a stylesheet - this must be stored in the `static` subdirectory

In [ ]:
mkdir -p static/styles
cat << EOF > static/styles/mystyles.css
.title {
    margin: 5px;
}

body {background-color: powderblue;}
table, th, td {
  border: 1px solid;
}
EOF

The project structure is now:

In [ ]:
tree -I __pycache__ .

Start the Flask development server:

In [ ]:
fuser -k 8000/tcp  # kill previous incarnations
flask --app database_app run --host localhost --port 8000 &

Now display the index view in firefox

In [ ]:
firefox http://localhost:8000

Now create a route to display a `single match`:

In [ ]:
cat << EOF >> database_app.py
@app.route('/single_match/<int:match_id>/')
def singleMatch(match_id):
    match = Match.query.get_or_404(match_id)
    return render_template('singleMatch.html', match=match, id=match_id)
EOF

Add the corresponding template (inheriting from the base template):

In [ ]:
cat << EOF > templates/singleMatch.html
{% extends 'base.html' %}
{% block title %}
    MATCH {{ id }}
{% endblock %}
{% block contents %}
    <div class="content">
    <div>{{ match }}</div>
{% endblock %}
EOF

Restart the server:

In [ ]:
fuser -k 8000/tcp  # kill previous incarnations
flask --app database_app run --host localhost --port 8000 &

Display the 2nd entry in the browser:

In [ ]:
firefox http://localhost:8000/single_match/2

Add a route to allow us to `create` new entries in the database:

In [ ]:
cat << EOF >> database_app.py
@app.route('/create/', methods=('GET', 'POST'))
def create():
    if request.method == 'POST':
        homeTeam = request.form['homeTeam']
        awayTeam = request.form['awayTeam']
        homeScore = request.form['homeScore']
        awayScore = request.form['awayScore']

        match = Match(homeTeam, homeScore, awayTeam, awayScore)
        db.session.add(match)
        db.session.commit()

        return redirect(url_for('index'))

    return render_template('create.html')
EOF

Add the template for the route:

In [ ]:
cat << EOF > templates/create.html
{% extends 'base.html' %}

{% block title %}
    <h1>Add Match {{ id }}</h1>
{% endblock %}

{% block contents %}
    <form method="post">
        <p>
            <label for="homeTeam">Home Team</label>
            <input id="h" type="text" name="homeTeam" placeholder="home team"/>
        </p>
        <p>
            <label for="homeScore">Home Score</label>
            <input type="text" name="homeScore" placeholder="home score"/>
        </p>
        <p>
            <label for="awayTeam">Away Team</label>
            <input type="text" name="awayTeam" placeholder="away team"/>
        </p>
        <p>
            <label for="awayScore">Away Score</label>
            <input type="text" name="awayScore" placeholder="away score"/>
        </p>
        <p>
            <button id="submit" type="submit">Submit</button>
        </p>
    </form>
{% endblock %}
EOF

Restart the server:

In [ ]:
fuser -k 8000/tcp  # kill previous incarnations
flask --app database_app run --host localhost --port 8000 &

In [ ]:
firefox http://localhost:8000/create

Now create a `Selenium` script to automate filling in the `create` form: 

In [ ]:
cat << EOF > go.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

options = webdriver.FirefoxOptions()
service = webdriver.FirefoxService(executable_path="/snap/bin/geckodriver")
driver = webdriver.Firefox(options=options, service=service)

driver.get("http://localhost:8000/create")

time.sleep(5)
title = driver.title

driver.implicitly_wait(0.5)

def fillInForm(h, hs, a, awayScore):
    time.sleep(5)
    text_box = driver.find_element(by=By.NAME, value="homeTeam")
    text_box.send_keys(h)
    text_box = driver.find_element(by=By.NAME, value="homeScore")
    text_box.send_keys(hs)
    text_box = driver.find_element(by=By.NAME, value="awayTeam")
    text_box.send_keys(a)
    text_box = driver.find_element(by=By.NAME, value="awayScore")
    text_box.send_keys(awayScore)
    driver.implicitly_wait(5)
    time.sleep(2)
    current_url = driver.current_url
    submit_button = driver.find_element(by=By.ID, value="submit")
    submit_button.click()    
    WebDriverWait(driver, 15).until(EC.url_changes(current_url))
    time.sleep(2)

def getResults():
    link = driver.find_element(By.LINK_TEXT, 'create')
    link.click()

fillInForm("White", 5, "Black", 3);getResults()
fillInForm("Purple", 4, "Red", 2);getResults()
fillInForm("Green", 7, "White", 0);getResults()
fillInForm("Blue", 4, "Yellow", 1);getResults()
fillInForm("Green", 5, "Blue", 0)
time.sleep(5)

driver.quit()
EOF

Now run the Selenium script to automatically fill in the `create` form.  
Note we need to define an accessible temporary directory for this to work:

In [ ]:
export TMPDIR=/tmp
python go.py

Clean up:
* remove workspace
* shutdown server

In [ ]:
cd ..
fuser -k 7000/tcp
rm -r workspace3